# Imports

In [152]:
import pandas as pd
import os 
current_folder = os.getcwd()
print(current_folder)
parent_folder = os.path.dirname(current_folder)
os.chdir(parent_folder)

/home/araceli/code/energy-analysis-and-predictions


In [162]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from utils.parameters import ENERGY_COMPANIES_FOLDER, ENERGY_COMPANY_SECTOR,\
                            DICT_COMPANIES_NAME_TO_CHANGE, \
                            LIST_WEIRD_FIRST_RAW

from load_data.load_energy_produced_by_companies import load_company_data_by_sector

from predict_data.create_train_test_data import get_scaled_X_y_train_and_test_data
from predict_data.create_simple_train_test_data import get_X_y_scaled_and_splited                       
from predict_data.load_gru_model import load_gru_model          
from predict_data.model_methods import compile_model, train_model,evaluate_model, predict_model                                          

from visualize_data.plot_price_with_interactive_timestamps import plot_price_with_interactive_timestamps   
from visualize_data.plot_model_history import plot_history
from visualize_data.plot_predict_vs_test import plot_predict_vs_test

import plotly.express as px
import plotly.graph_objects as go

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Company Sectors

In [120]:
energy_sectors = pd.read_csv(ENERGY_COMPANY_SECTOR , delimiter = ",", encoding = "utf-8") 
energy_sectors['Energy_Source'] = energy_sectors['Energy_Source'].map(lambda x: str(x)[:-1])
energy_sectors['Name'] = energy_sectors['Name'].map(lambda x: str(x)[:-1])
energy_sectors['Name'] = energy_sectors['Name'].map(lambda x: str(x).replace(' ', '-'))
energy_sectors['Name'] = energy_sectors['Name'].map(lambda x: str(x).replace('_', '-'))
energy_sectors = energy_sectors.dropna()
energy_sectors = energy_sectors.drop_duplicates()
energy_sectors.head()

,Name,Place,Energy_Source,Rated_capacity
0,Abfallentsorgungszentrum-Asdonkshof,Kamp-Lintfort,Abfall,20.8 MW
1,Abfallheizkraftwerk-Neunkirchen,Neunkirchen,Abfall,11.6 MW
2,AMK---Abfallentsorgungsgesellschaft-des-Maerki...,Iserlohn,Abfall,12.6 MW
3,AVA-GmbH,Augsburg,Abfall,10 MW
4,AVA-Velsen,Saarbruecken,Abfall,16 MW


## Test company is included in sectors

In [121]:
cond = energy_sectors['Name'].str.contains('Witznau') 
#Trianel_Gaskraftwerk
# Vorarlberger_Illwerke_AG__Obervermuntwerk_II_
#cond = energy_sectors['Energy_Source'] == 'Laufwasser'
#cond = energy_sectors['Name'] == 'Boxberg'
#test = energy_sectors[cond].sort_values(by='Name')
#test['Name'][532]
test = energy_sectors[cond]
#test['Name'].values
test

,Name,Place,Energy_Source,Rated_capacity
640,Witznau,Witznau,Pumpspeicher,220 MW


In [284]:
cond2 = energy_sectors['Energy_Source']=='Wind (Onshore)' 
#cond = energy_sectors['Energy_Source'] == 'Laufwasser'
#cond = energy_sectors['Name'] == 'Boxberg'
#test = energy_sectors[cond].sort_values(by='Name')
#test['Name'][532]
test2 = energy_sectors[cond2]
test2.reset_index(drop=True, inplace=True)
test2[80:]

,Name,Place,Energy_Source,Rated_capacity
80,Windpark-Weimar,Bucha,Wind (Onshore),12 MW
81,Windpark-Weilrod,Weilrod,Wind (Onshore),"16,8 MW"
82,Windpark-Weener-Verwaltungs-GmbH,Weener,Wind (Onshore),"21,5 MW"
83,Windpark-Wangelau,Wangelau,Wind (Onshore),"14,4 MW"
84,Windpark-Wanderup-West-GmbH-&-Co.-KG,Wanderup,Wind (Onshore),"15,9 MW"
85,Windpark-Wanderup,Wanderup,Wind (Onshore),"34,5 MW"
86,Windpark-Walsleben-Goldbeck,Goldbeck,Wind (Onshore),"18,3 MW"
87,Windpark-Walsleben-Goldbeck,Hansestadt Osterburg,Wind (Onshore),"21,3 MW"
88,Windpark-Waldow,Waldow,Wind (Onshore),28 MW


## Dictionary with companies to change

In [5]:
text='Heizkraftwerk_R_merbr_cke'
if text in DICT_COMPANIES_NAME_TO_CHANGE.keys():
    text=DICT_COMPANIES_NAME_TO_CHANGE[text]
print(text)

Heizkraftwerk-Roemerbruecke


## Energy sector types

In [6]:
energy_sector_types = energy_sectors['Energy_Source'].value_counts()
energy_sector_types

Erdgas                                    158
Laufwasser                                104
Wind (Onshore)                             89
Steinkohle                                 64
Photovoltaik                               63
Biomasse                                   49
Abfall                                     43
Pumpspeicher                               29
Wind (Offshore)                            28
Braunkohle                                 26
Speicherwasser (ohne Pumpspeicher)         23
Sonstige konventionelle Energietraeger     22
Mineraloelprodukte                         22
Batteriespeicher                           15
Kernenergie                                 8
Steinkohle - Heizoe                         1
Erdgas – Heizoe                             1
Waerme                                      1
Name: Energy_Source, dtype: int64

In [7]:
cond = energy_sectors['Name'].str.contains('Abwinden')
#cond = energy_sectors['Name'] == ' Abwinden-Asten'
energy_sectors[cond]

,Name,Place,Energy_Source,Rated_capacity
533,Abwinden-Asten,Luftenberg/Asten,Laufwasser,168 MW


In [64]:
dict_companies = {sector:[] for sector in sectors}
dict_companies

{'Abfall': [],
 'Batteriespeicher': [],
 'Biomass': [],
 'Biomasse': [],
 'Braunkohle': [],
 'Erdgas': [],
 'Erdgas – Heizoe': [],
 'Kernenergie': [],
 'Laufwasser': [],
 'Mineraloelprodukte': [],
 'Photovoltaik': [],
 'Pumpspeicher': [],
 'Sonstige konventionelle Energietraeger': [],
 'Speicherwasser (ohne Pumpspeicher)': [],
 'Steinkohle': [],
 'Steinkohle - Heizoe': [],
 'Waerme': [],
 'Wind (Offshore)': [],
 'Wind (Onshore)': [],
 'na': []}

# Power Plant data

In [9]:
#foldername = os.path.join(ENERGY_COMPANIES_FOLDER,'power-plant-data-2022-Jan-to-Dec' )
power_plant_list = os.listdir(ENERGY_COMPANIES_FOLDER)
power_plant_list.sort()
print('len(power_plant_list)=', len(power_plant_list))


len(power_plant_list)= 221


In [10]:
for file in power_plant_list:
    if 'Identifier' in file:
        power_plant_list.remove(file)
        
len(power_plant_list)

112

## Change delimiter

In [11]:
name ='Kraftwerk_BASF_Ludwigshafen_Mitte'
index=None
for file in power_plant_list:
    if name in file:
        index = power_plant_list.index(file)       
        print(index)

In [12]:
filename = os.path.join(ENERGY_COMPANIES_FOLDER, power_plant_list[index])
print(filename)
df = pd.read_csv(filename, delimiter=';')
#df.to_csv(filename, sep=';', index=False)
df.head()

TypeError: list indices must be integers or slices, not NoneType

In [303]:
df = pd.read_csv(filename, delimiter=';', decimal=',')
df['Generation_DE Koepchenwerk [MW] Berechnete Auflösungen'].isna().value_counts()/df.shape[0]

False    1.0
Name: Generation_DE Koepchenwerk [MW] Berechnete Auflösungen, dtype: float64

In [306]:
type(df['Generation_DE Koepchenwerk [MW] Berechnete Auflösungen'][0])

str

In [156]:
if '(3 GT + 1 DT, Sammelschiene)' in df['Datum'][0]:
    df= df.iloc[1:,:]
df.head()

,Datum,Anfang,Ende,Generation_DE Heizkraftwerk Dresden-Nossener Brücke
1,01.01.2022,00:00,00:00,1.284
2,02.01.2022,00:00,00:00,1.354
3,03.01.2022,00:00,00:00,2.328
4,04.01.2022,00:00,00:00,4.449
5,05.01.2022,00:00,00:00,3.913


In [259]:
if type(df['Datum'][0])==str:
    df = df.iloc[1:,:]
    df.reset_index(drop=True, inplace=True)
df.head()

,Datum,Anfang,Ende,Generation_DE Heizkraftwerk Dresden-Nossener Brücke
0,01.01.2022,00:00,00:00,1.284
1,02.01.2022,00:00,00:00,1.354
2,03.01.2022,00:00,00:00,2.328
3,04.01.2022,00:00,00:00,4.449
4,05.01.2022,00:00,00:00,3.913


In [256]:
#type(df['Datum'][0])
df.columns

Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE Heizkraftwerk Dresden-Nossener Brücke '],
      dtype='object')

In [13]:
def clean_column_dataframe(text):
    text = str(text)
    text = text.replace('.', '')
    text = text.replace(',', '.')
    text = text.replace('-', np.nan)
    text = text.astype('float32')
    text = text/1000 # true scale
    return text

In [262]:
df['new']=df['Generation_DE Heizkraftwerk Dresden-Nossener Brücke '].apply(clean_column_dataframe)
df

TypeError: replace() argument 2 must be str, not float

In [114]:
cols = list(df.columns)
for col in cols:
    print(col)
    if 'Unnamed' in col:
        df=df.drop(columns=col)
df.head()
    

Datum
Anfang
Ende
Generation_DE Block 4 [MW] Berechnete Auflösungen
Unnamed: 4


,Datum,Anfang,Ende,Generation_DE Block 4 [MW] Berechnete Auflösungen
0,01.01.2022,00:00,00:00,0
1,02.01.2022,00:00,00:00,0
2,03.01.2022,00:00,00:00,0
3,04.01.2022,00:00,00:00,0
4,05.01.2022,00:00,00:00,0


In [ ]:
def convert_str_to_float(text):
    if text == '-':
        return 0
    else:
        text=text.replace(',','.')
        return float(text)

# Test load all the companis by sectors 

In [163]:
#DICT_COMPANIES_NAME_TO_CHANGE
# LIST_WEIRD_FIRST_RAW

In [188]:
dict_companies = load_company_data_by_sector()

Abwinden_Asten_202201010000_202212312359_Tag_1.csv
2 company =Abwinden-Asten
Laufwasser
*************************************************************************
Altenwoerth_202201010000_202212312359_Tag_3.csv
2 company =Altenwoerth
Laufwasser
*************************************************************************
Bergkamen_202201010000_202212312359_Tag_6.csv
2 company =Bergkamen
Steinkohle
*************************************************************************
Boxberg_202201010000_202212312359_Tag_8.csv
2 company =Boxberg
Braunkohle
*************************************************************************
Braunkohlekraftwerk_Lippendorf_202201010000_202212312359_Tag_9.csv
2 company =Braunkohlekraftwerk-Lippendorf
Braunkohle
*************************************************************************
Burghausen_GT_202201010000_202212312359_Tag_12.csv
2 company =Burghausen-GT
Erdgas
*************************************************************************
Cuno_Heizkraftwerk_Herdecke_202

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_

Kernenergie
*************************************************************************
Gerlos_202201010000_202212312359_Tag_43.csv
2 company =Gerlos
Speicherwasser (ohne Pumpspeicher)
*************************************************************************
Gersteinwerk_202201010000_202212312359_Tag_44.csv
2 company =Gersteinwerk
Steinkohle
*************************************************************************
Goldisthal_202201010000_202212312359_Tag_46.csv
2 company =Goldisthal
Pumpspeicher
*************************************************************************
Greifenstein_202201010000_202212312359_Tag_47.csv
2 company =Greifenstein
Laufwasser
*************************************************************************
Gro_kraftwerk_Mannheim_202201010000_202212312359_Tag_49.csv
1 company =Grosskraftwerk-Mannheim.
2 company =Grosskraftwerk-Mannheim
Steinkohle
*************************************************************************
Gro_kraftwerk_Mannheim_202201010000_202212312359_Tag

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_

2 company =Heizkraftwerk-Niehl
not the right number of columns
Index(['Datum', 'Anfang', 'Ende'], dtype='object')
not added to dict
*************************************************************************
Heizkraftwerk_R_merbr_cke_202201010000_202212312359_Tag_69.csv
1 company =Heizkraftwerk-Roemerbruecke.
2 company =Heizkraftwerk-Roemerbruecke
Erdgas
*************************************************************************
Heizkraftwerk_West_Wolfsburg_202201010000_202212312359_Tag_72.csv
2 company =Heizkraftwerk-West-Wolfsburg
Steinkohle
*************************************************************************
Heizkraftwerk_West__Frankfurt__202201010000_202212312359_Tag_71.csv
1 company =Heizkraftwerk-West-(Frankfurt).
2 company =Heizkraftwerk-West-(Frankfurt)
Erdgas
*************************************************************************
Heyden_202201010000_202212312359_Tag_74.csv
2 company =Heyden
not the right number of columns
Index(['Datum', 'Anfang', 'Ende'], dtype='object')
n

...

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_

2 company =Ottensheim-Wilhering
Laufwasser
*************************************************************************
Pumpspeicherkraftwerk_Vianden_202201010000_202212312359_Tag_137.csv
2 company =Pumpspeicherkraftwerk-Vianden
Pumpspeicher
*************************************************************************
Restm_ll-Heizkraftwerk_Stuttgart-M_nster_202201010000_202212312359_Tag_138.csv
1 company =Restmuell-Heizkraftwerk-Stuttgart-Muenster.
2 company =Restmuell-Heizkraftwerk-Stuttgart-Muenster
Steinkohle
*************************************************************************
Reuter_West_202201010000_202212312359_Tag_140.csv
2 company =Reuter-West
Steinkohle
*************************************************************************
Rheinhafen-Dampfkraftwerk_202201010000_202212312359_Tag_141.csv
2 company =Rheinhafen-Dampfkraftwerk
Steinkohle
*************************************************************************
Rheinkraftwerk_Iffezheim_202201010000_202212312359_Tag_142.csv
2 compa

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_

2 company =Trianel-Gaskraftwerk
Erdgas
*************************************************************************
Trianel_Kohlekraftwerk_L_nen_202201010000_202212312359_Tag_157.csv
1 company =Trianel-Kohlekraftwerk-Luenen.
2 company =Trianel-Kohlekraftwerk-Luenen
Steinkohle
*************************************************************************
V_lklingen_202201010000_202212312359_Tag_158.csv
1 company =Voelklingen.
2 company =Voelklingen
Steinkohle
*************************************************************************
Vorarlberger_Illwerke_AG__Obervermuntwerk_II__202201010000_202212312359_Tag_159.csv
1 company =Vorarlberger-Illwerke-AG-"Obervermuntwerk-II".
2 company =Vorarlberger-Illwerke-AG-"Obervermuntwerk-II"
Pumpspeicher
*************************************************************************
Waldeck_2_202201010000_202212312359_Tag_160.csv
2 company =Waldeck-2
Pumpspeicher
*************************************************************************
Wallsee-Mitterkirchen_2022010

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_by_companies.py:89: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/home/araceli/code/energy-analysis-and-predictions/load_data/load_energy_produced_

In [126]:
# Empty data csv files:

# Heizkraftwerk_Merkenich_202201010000_202212312359_Tag_63.csv
# 2 company =Heizkraftwerk-Merkenich
# not the right number of columns
# Erdgas
# *************************************************************************
# Heizkraftwerk_Niehl_202201010000_202212312359_Tag_67.csv
# 2 company =Heizkraftwerk-Niehl
# not the right number of columns
# Erdgas

# Heyden_202201010000_202212312359_Tag_74.csv
# 2 company =Heyden
# not the right number of columns
# Steinkohle


# KWM_Mehrum_202201010000_202212312359_Tag_120.csv
# 2 company =KWM-Mehrum
# not the right number of columns
# Steinkohle

# Kraftwerk_Silz_202201010000_202212312359_Tag_110.csv
# 2 company =Kraftwerk-Silz
# not the right number of columns
# Speicherwasser (ohne Pumpspeicher)


# Thei__202201010000_202212312359_Tag_152.csv
# 1 company =Theiss.
# 2 company =Theiss
# not the right number of columns
# Erdgas

['Abfall',
 'Batteriespeicher',
 'Biomass',
 'Biomasse',
 'Braunkohle',
 'Erdgas',
 'Erdgas – Heizoe',
 'Kernenergie',
 'Laufwasser',
 'Mineraloelprodukte',
 'Photovoltaik',
 'Pumpspeicher',
 'Sonstige konventionelle Energietraeger',
 'Speicherwasser (ohne Pumpspeicher)',
 'Steinkohle',
 'Steinkohle - Heizoe',
 'Waerme',
 'Wind (Offshore)',
 'Wind (Onshore)']

In [150]:
df_test_1 = pd.read_csv('./raw_data/energy-companies/power-plant-data-2022-Jan-to-Dec/Heizkraftwerk_Merkenich_202201010000_202212312359_Tag_63.csv', delimiter=';', decimal=',')
df_test_1.head()

,Datum,Anfang,Ende,Generation_DE Block 4 [MW] Berechnete Auflösungen,Generation_DE Block 6 [MW] Berechnete Auflösungen,Generation_DE Gas-und-Dampfturbinen [MW] Berechnete Auflösungen
0,01.01.2022,00:00,00:00,-,-,-
1,02.01.2022,00:00,00:00,-,-,-
2,03.01.2022,00:00,00:00,-,-,-
3,04.01.2022,00:00,00:00,-,-,-
4,05.01.2022,00:00,00:00,-,-,-


In [151]:
df_test_1.tail()

,Datum,Anfang,Ende,Generation_DE Block 4 [MW] Berechnete Auflösungen,Generation_DE Block 6 [MW] Berechnete Auflösungen,Generation_DE Gas-und-Dampfturbinen [MW] Berechnete Auflösungen
360,27.12.2022,00:00,00:00,-,-,-
361,28.12.2022,00:00,00:00,-,-,-
362,29.12.2022,00:00,00:00,-,-,-
363,30.12.2022,00:00,00:00,-,-,-
364,31.12.2022,00:00,00:00,-,-,-


# Analyse different sector

In [161]:
len(dict_companies['Laufwasser'])

11

In [179]:
dict_companies['Laufwasser'][2].columns

Index(['Date', 'Start', 'End', 'Greifenstein [MW]'], dtype='object')

In [189]:
test =  dict_companies['Laufwasser'][2].copy()
#test =
test.set_index('Date', inplace=True)
test.head()

,Start,End,Greifenstein [MW]
Date,,,
2022-01-01,00:00,00:00,7267.0
2022-01-02,00:00,00:00,6714.0
2022-01-03,00:00,00:00,5971.0
2022-01-04,00:00,00:00,5726.0
2022-01-05,00:00,00:00,5999.0


In [173]:
test.columns[2]

'Greifenstein [MW]'

In [192]:
fig = go.Figure()

for company in dict_companies['Laufwasser']:
    target = company.columns[3]
    print(target)
    print(company.columns)   
    #company = company.set_index('Date', inplace=True)
    fig.add_trace(go.Scatter(x=company['Date'], y=company[target], name = target))

fig.show()

Abwinden-Asten [MW]
Index(['Date', 'Start', 'End', 'Abwinden-Asten [MW]'], dtype='object')
Altenwoerth [MW]
Index(['Date', 'Start', 'End', 'Altenwoerth [MW]'], dtype='object')
Greifenstein [MW]
Index(['Date', 'Start', 'End', 'Greifenstein [MW]'], dtype='object')
Kraftwerk-Rheinfelden [MW]
Index(['Date', 'Start', 'End', 'Kraftwerk-Rheinfelden [MW]'], dtype='object')
Laufenburg [MW]
Index(['Date', 'Start', 'End', 'Laufenburg [MW]'], dtype='object')
Melk [MW]
Index(['Date', 'Start', 'End', 'Melk [MW]'], dtype='object')
Ottensheim-Wilhering [MW]
Index(['Date', 'Start', 'End', 'Ottensheim-Wilhering [MW]'], dtype='object')
Rheinkraftwerk-Iffezheim [MW]
Index(['Date', 'Start', 'End', 'Rheinkraftwerk-Iffezheim [MW]'], dtype='object')
Wallsee-Mitterkirchen [MW]
Index(['Date', 'Start', 'End', 'Wallsee-Mitterkirchen [MW]'], dtype='object')
Wien-Freudenau [MW]
Index(['Date', 'Start', 'End', 'Wien-Freudenau [MW]'], dtype='object')
Ybbs-Persenbeug [MW]
Index(['Date', 'Start', 'End', 'Ybbs-Persenbeug

In [193]:
fig = go.Figure()

for company in dict_companies['Erdgas']:
    target = company.columns[3]
    print(target)
    print(company.columns)   
    #company = company.set_index('Date', inplace=True)
    fig.add_trace(go.Scatter(x=company['Date'], y=company[target], name = target))

fig.show()

Burghausen-GT [MW]
Index(['Date', 'Start', 'End', 'Burghausen-GT [MW]'], dtype='object')
Cuno-Heizkraftwerk-Herdecke [MW]
Index(['Date', 'Start', 'End', 'Cuno-Heizkraftwerk-Herdecke [MW]'], dtype='object')
Donaustadt [MW]
Index(['Date', 'Start', 'End', 'Donaustadt [MW]'], dtype='object')
Dormagen [MW]
Index(['Date', 'Start', 'End', 'Dormagen [MW]'], dtype='object')
Duisburg-Heizkraftwerk-III [MW]
Index(['Date', 'Start', 'End', 'Duisburg-Heizkraftwerk-III [MW]'], dtype='object')
Emsland [MW]
Index(['Date', 'Start', 'End', 'Emsland [MW]'], dtype='object')
Franken-1 [MW]
Index(['Date', 'Start', 'End', 'Franken-1 [MW]'], dtype='object')
Gas-und-Dampfturbinen-Kraftwerk-Mitte [MW]
Index(['Date', 'Start', 'End', 'Gas-und-Dampfturbinen-Kraftwerk-Mitte [MW]'], dtype='object')
Gaskraftwerk-Irsching [MW]
Index(['Date', 'Start', 'End', 'Gaskraftwerk-Irsching [MW]'], dtype='object')
GuD-Marzahn [MW]
Index(['Date', 'Start', 'End', 'GuD-Marzahn [MW]'], dtype='object')
Heizkraftwerk-Altbach/Deizisau [